In [1]:
from datasets_shim import *
from batched_dataloader import get_batched_dataloader, BatchEnum
from embedding_inference import EmbeddingInference

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed')).History will not be written to the database.


/home/nikola.jovisic.ivi/.local/lib/python3.10/site-packages/_distutils_hack/__init__.py:53: UserWarning: Reliance on distutils from stdlib is deprecated. Users must rely on setuptools to provide the distutils module. Avoid importing distutils or import setuptools first, and avoid setting SETUPTOOLS_USE_DISTUTILS=stdlib. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(
/home/nikola.jovisic.ivi/nj/mammo-filter/model/layers/swiglu_ffn.py:43: UserWarning: xFormers is available (SwiGLU)
  warnings.warn("xFormers is available (SwiGLU)")
/home/nikola.jovisic.ivi/nj/mammo-filter/model/layers/attention.py:27: UserWarning: xFormers is available (Attention)
  warnings.warn("xFormers is available (Attention)")
/home/nikola.jovisic.ivi/nj/mammo-filter/model/layers/block.py:33: UserWarning: xFormers is available (Block)
  warnings.warn("xFormers is available (Block)")


In [16]:
import torch
import torch.nn as nn
import torch.optim as optim
import h5py
import numpy as np
from torch.utils.data import Dataset, DataLoader
from collections import defaultdict
from sklearn.metrics import (
    accuracy_score, balanced_accuracy_score, precision_score,
    recall_score, f1_score, roc_auc_score, confusion_matrix
)
from sklearn.metrics import classification_report

#############

# UPOREDI DA LI BOLJE RADI KAD SE TRENIRA NA SVAKOJ SLICI POSEBNO, PA RADI MAXPOOLING, ILI KAD SE TRENIRA SA MAXPOOLINGOM KROZ SLIKE -> INICIJALNI DOKAZ DA TREBA RADITI MIL

############




def evaluate_metrics(model, loader, split):
    model.eval()
    all_logits = []
    all_labels = []
    with torch.no_grad():
        for x, y, _ in loader:
            x, y = x.to('cuda'), y.to('cuda')
            logits = model(x)
            all_logits.append(logits.cpu())
            all_labels.append(y.cpu())

    logits = torch.cat(all_logits).squeeze(1)
    labels = torch.cat(all_labels).squeeze(1)

    probs = torch.sigmoid(logits).numpy()
    preds = (probs > 0.5).astype(int)
    targets = labels.numpy().astype(int)

    cm = confusion_matrix(targets, preds)
    row_sums = cm.sum(axis=1, keepdims=True)
    balanced_cm = cm / row_sums

#     print(f'\n{split} Confusion Matrices:')
#     print(f'  Regular Confusion Matrix:\n{cm}')
#     print(f'  Balanced Confusion Matrix (rows sum to 1):\n{balanced_cm.round(4)}')

    return np.diag(balanced_cm)


def linear_probe(train_path, valid_path, test_path, epochs=100, batch_size=64, lr=1e-3, patience=5):
    train_dataset = EmbeddingsDataset(train_path)
    valid_dataset = EmbeddingsDataset(valid_path)
    test_dataset = EmbeddingsDataset(test_path)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    valid_loader = DataLoader(valid_dataset, batch_size=batch_size)
    test_loader = DataLoader(test_dataset, batch_size=batch_size)

    model = nn.Linear(768, 1).to('cuda')
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-4)
    criterion = nn.BCEWithLogitsLoss(reduction='none')

    best_val_loss = float('inf')
    patience_counter = 0

    for epoch in range(epochs):
        model.train()
        total_loss = 0
        for x, y, w in train_loader:
            x, y, w = x.to('cuda'), y.to('cuda'), w.to('cuda')
            logits = model(x)
            loss = criterion(logits, y)
            loss = (loss * w).mean()

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        avg_train_loss = total_loss / len(train_loader)

        model.eval()
        val_loss = 0
        with torch.no_grad():
            for x, y, w in valid_loader:
                x, y, w = x.to('cuda'), y.to('cuda'), w.to('cuda')
                logits = model(x)
                loss = criterion(logits, y)
                loss = (loss * w).mean()
                val_loss += loss.item()
        avg_val_loss = val_loss / len(valid_loader)

        #print(f'Epoch {epoch+1}: Train Loss = {avg_train_loss:.4f}, Val Loss = {avg_val_loss:.4f}')

        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            best_model_state = model.state_dict()
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f'\nEarly stopping triggered after {epoch+1} epochs.')
                break

    model.load_state_dict(best_model_state)
    evaluate_metrics(model, valid_loader, 'Validation')
    test_diag = evaluate_metrics(model, test_loader, 'Test')

    return test_diag

diags = []
for i in range(20):
    print(f'\n=== Run {i+1}/20 ===')
    diag = linear_probe(
        '../lustre_mock/vindr-simple-train/embeddings.hdf5',
        '../lustre_mock/vindr-simple-test/embeddings.hdf5',
        '../lustre_mock/vindr-simple-valid/embeddings.hdf5'
    )
    diags.append(diag)

diags = np.stack(diags)
mean_diag = diags.mean(axis=0)
std_diag = diags.std(axis=0)

print('\nMean of balanced confusion matrix diagonal over 20 runs:')
print(mean_diag.round(4))

print('\nStandard deviation of balanced confusion matrix diagonal over 20 runs:')
print(std_diag.round(4))




=== Run 1/20 ===
464
96
92


/tmp/ipykernel_8176/1778411168.py:30: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  raw_label = int(group['label'][()])



=== Run 2/20 ===
464
96
92


/tmp/ipykernel_8176/1778411168.py:30: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  raw_label = int(group['label'][()])



=== Run 3/20 ===
464
96
92


...


=== Run 19/20 ===
464
96
92


/tmp/ipykernel_8176/1778411168.py:30: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  raw_label = int(group['label'][()])



=== Run 20/20 ===
464
96
92


/tmp/ipykernel_8176/1778411168.py:30: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  raw_label = int(group['label'][()])



Mean of balanced confusion matrix diagonal over 20 runs:
[0.5635 0.572 ]

Standard deviation of balanced confusion matrix diagonal over 20 runs:
[0.0279 0.0657]
